## Load Tempo Artifact and Call

In [1]:
import os
import numpy as np
from tempo.serve.loader import load
ARTIFACTS_FOLDER = os.getcwd()+"/artifacts"
model = load(f"{ARTIFACTS_FOLDER}/classifier")

In [2]:
model.details.inputs

ModelDataArgs(args=[ModelDataArg(ty=<class 'numpy.ndarray'>, name='payload')])

In [3]:
from tempo.serve.metadata import RuntimeOptions, KubernetesOptions
runtime_options = RuntimeOptions(
        runtime="tempo.seldon.SeldonKubernetesRuntime",
        k8s_options=KubernetesOptions(
            namespace="production",
            authSecretName="minio-secret"
        )
    )

In [4]:
model.set_runtime_options_override(runtime_options)

In [6]:
model.remote(np.array([[0, 0, 0,0]]))

{'output0': array([1.], dtype=float32), 'output1': 'sklearn prediction'}

Questions

 * Nicer way to set runtime_options?

### Call via Requests

In [13]:
from tempo.seldon.k8s import SeldonKubernetesRuntime
k8s_runtime = SeldonKubernetesRuntime(runtime_options)
url = k8s_runtime.get_endpoint(model)

In [14]:
payload = model.model_spec.protocol.to_protocol_request(np.array([[0, 0, 0,0]]))

In [15]:
import requests
res = requests.post(url,json=payload)
print(res)
print(res.json())

<Response [200]>
{'model_name': 'classifier', 'model_version': None, 'id': '9bb1d795-e941-44ef-97a9-ec8cf6322b92', 'parameters': None, 'outputs': [{'name': 'output0', 'shape': [1], 'datatype': 'FP32', 'parameters': None, 'data': [1.0]}, {'name': 'output1', 'shape': [18], 'datatype': 'BYTES', 'parameters': None, 'data': [115, 107, 108, 101, 97, 114, 110, 32, 112, 114, 101, 100, 105, 99, 116, 105, 111, 110]}]}


Questions:
 * We need a better way to get core details or URL and Payload so one can run an external curl/requests oneself?

## Create Tempo Model from Metadata


In [27]:
from tempo.serve.model import Model
from typing import Tuple
from tempo.serve.metadata import ModelFramework
from tempo.kfserving.protocol import KFServingV2Protocol

model2 = Model(name=model.details.name, 
               platform=ModelFramework.Custom,
               inputs={"payload":np.ndarray}, 
               outputs=(np.ndarray, str),
               protocol=KFServingV2Protocol(),
               runtime_options=runtime_options)

In [28]:
model2.remote(np.array([[0, 0, 0,0]]))

{'output0': array([1.], dtype=float32), 'output1': 'sklearn prediction'}

Questions

 * For this we would need a `serialize` and `deserialize` method to store the metadata for the model?